In [2]:
import gym
import random
from time import sleep
import numpy as np
from IPython.display import clear_output # caso você não usa o Jupyer ou o Colab, pode comentar ou apagar essa linha de código
!pip install pygame

In [11]:
env = gym.make('Taxi-v3').env
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [12]:
# 0 = south, 1 = north, 2 = east, 3 = west, 4 = pickup and 5 = dropoff

print(env.action_space)

Discrete(6)


In [13]:
print(env.observation_space)

Discrete(500)


In [14]:
len(env.P)

500

In [16]:
env.P[484]

{0: [(1.0, 484, -1, False)],
 1: [(1.0, 384, -1, False)],
 2: [(1.0, 484, -1, False)],
 3: [(1.0, 464, -1, False)],
 4: [(1.0, 484, -10, False)],
 5: [(1.0, 484, -10, False)]}

In [19]:
env.P[384]

{0: [(1.0, 484, -1, False)],
 1: [(1.0, 284, -1, False)],
 2: [(1.0, 384, -1, False)],
 3: [(1.0, 364, -1, False)],
 4: [(1.0, 384, -10, False)],
 5: [(1.0, 384, -10, False)]}

In [31]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table.shape

(500, 6)

In [32]:
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [33]:
np.argmax(np.array([3, 5]))

1

In [34]:
%%time

alpha = 0.1
gamma = 0.6
epsilon = 0.1

for i in range(100000):
    estado = env.reset()
    
    penalidades, recompensa = 0, 0
    done = False
    
    while not done:
        # exploracao
        if random.uniform(0, 1) < epsilon:
            acao = env.action_space.sample()
            # exploitation
        else:
            acao = np.argmax(q_table[estado])
        
        proximo_estado, recompensa, done, info = env.step(acao)
        
        q_antigo = q_table[estado, acao]
        proximo_maximo = np.max(q_table[proximo_estado])
        
        q_novo = (1 - alpha) * q_antigo + alpha * (recompensa + gamma * proximo_maximo)
        q_table[estado, acao] = q_novo
        
        if recompensa == -10:
            penalidades += 1
            
        estado = proximo_estado
    
    if i % 100 == 0:
        clear_output(wait=True)
        print(f'Episódio {i}')
        
print('Treinamento concluído')

Episódio 99900
Treinamento concluído
Wall time: 1min 44s


In [36]:
q_table[424]

array([-2.49476939, -2.4936525 , -2.4936525 , -2.4948838 , -6.38305771,
       -9.04335906])

In [81]:
env.reset()
env.render()

# azul, ponto de partida
# roxo, destino final

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [82]:
env.encode(2, 4, 0, 3)

283

In [83]:
q_table[283]

array([ -2.49136998,  -2.49164532,  -2.48785148,  -2.48236806,
       -10.79594645, -10.02539666])

In [85]:
total_penalidades = 0
episodios = 50
frames = []

for _ in range(episodios):
    estado = env.reset()
    penalidades, recompensa = 0, 0
    done = False
    
    while not done:
        acao = np.argmax(q_table[estado])
        estado, recompensa, done, info = env.step(acao)
        
        if recompensa == -10:
            penalidades += 1
            
        frames.append({'frame': env.render(mode='ansi'),
                      'state': estado,
                       'action': acao,
                       'reward': recompensa
                      })
        
        total_penalidades += penalidades
        
print(f'Episódios {episodios}')
print(f'Penalidades {total_penalidades}')

Episódios 50
Penalidades 0


In [87]:
frames[0]

{'frame': '+---------+\n|R: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n|\x1b \x1b| : | : |\n|\x1bY\x1b| : |B: |\n+---------+\n  (South)\n',
 'state': 309,
 'action': 0,
 'reward': -1}

In [94]:
for frame in frames:
    clear_output(wait=True)
    print(frame['frame'])
    print(f"Estado {frame['state']}")
    print(f"Ação {frame['action']}")
    print(f"Recompensa {frame['reward']}")
    sleep(0.5)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

Estado 476
Ação 4
Recompensa -1


KeyboardInterrupt: 